In [ ]:
import gzip
import os
from collections import defaultdict
import random
import pandas as pd
from sklearn import linear_model
import dateutil.parser
from datetime import timedelta
import csv
import pickle
import numpy as np

path = "./"
os.chdir(path)

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [4]:
places_data = list(parse("./data/places.clean.json.gz"))
user_data = list(parse("./data/users.clean.json.gz"))
review_data = list(parse("./data/reviews.clean.json.gz"))

print("done")

done


In [5]:
print(user_data[0])
#userName : string
#jobs : nested list [['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật XD', [[1, 1, 1998], [1, 1, 2013], 1], '', '']]
#currentPlace : ['Thành phố Hồ Chí Minh, Việt Nam', [[], 108230990, 1066296640, 1]]
#previousPlaces : [['tp. Nam Định, Nam Định, Việt Nam', [[], 204200000, 1061683330, 1]]]
#education : nested list
#gPlusUserId : string

{'userName': 'an lam', 'jobs': [['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật XD', [[1, 1, 1998], [1, 1, 2013], 1], '', '']], 'currentPlace': ['Thành phố Hồ Chí Minh, Việt Nam', [[], 108230990, 1066296640, 1]], 'previousPlaces': [['tp. Nam Định, Nam Định, Việt Nam', [[], 204200000, 1061683330, 1]]], 'education': [[[], [], [], [], [], 6], [['Đại học Kiến trúc tp Hồ chí minh, Việt nam', 'KS.XD', [[], [1, 1, 2013], 1], '', '']]], 'gPlusUserId': '100000010817154263736'}


In [6]:
print(places_data[0])
#attributes of places_data
#name : string
#price : None /
#address : list(strings)
#hours : [['Monday', [['6:30 am--4:15 pm']], ['Tuesday', [['6:30 am--4:15 pm']]]]
#phone : string
#gPlusPlaceId : string
#gps: [float,float]

{'name': 'Diamond Valley Lake Marina', 'price': None, 'address': ['2615 Angler Ave', 'Hemet, CA 92545'], 'hours': [['Monday', [['6:30 am--4:15 pm']]], ['Tuesday', [['6:30 am--4:15 pm']]], ['Wednesday', [['6:30 am--4:15 pm']], 1], ['Thursday', [['6:30 am--4:15 pm']]], ['Friday', [['6:30 am--4:15 pm']]], ['Saturday', [['6:30 am--4:15 pm']]], ['Sunday', [['6:30 am--4:15 pm']]]], 'phone': '(951) 926-7201', 'closed': False, 'gPlusPlaceId': '104699454385822125632', 'gps': [33.703804, -117.003209]}


In [7]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
state_list = set(state_list)

In [8]:
req_places = []
for d in places_data:
  t = d["address"][-1].split()
  for ad in t:
    if ad in state_list:
      req_places.append(d)

In [ ]:
#with open("USA_places.pkl","wb") as fp:   
#  pickle.dump(req_places,fp)

#with open("USA_places.pkl","rb") as fp:
#  req_places = pickle.load(fp)

In [9]:
req_places[100000]

{'name': 'Office Depot',
 'price': '$$',
 'address': ['1406 W Expy 83', 'Weslaco, TX 78596'],
 'hours': [['Monday', [['8:00 am--9:00 pm']]],
  ['Tuesday', [['8:00 am--9:00 pm']]],
  ['Wednesday', [['8:00 am--9:00 pm']], 1],
  ['Thursday', [['8:00 am--9:00 pm']]],
  ['Friday', [['8:00 am--9:00 pm']]],
  ['Saturday', [['9:00 am--9:00 pm']]],
  ['Sunday', [['10:00 am--7:00 pm']]]],
 'phone': '(956) 968-6207',
 'closed': False,
 'gPlusPlaceId': '103560782244834035321',
 'gps': [26176207.0, -98024823.0]}

In [10]:
place_ids = [d['gPlusPlaceId'] for d in req_places]
place_id_set = set(place_ids)

In [11]:
review_data[2000]

{'rating': 4.0,
 'reviewerName': 'Mitesh Shanbhag',
 'reviewText': 'A churrascaria with a good selection of salads and meats. They offer rodízio service, but sadly no red/green cards. Also strange that the whole seating area is in a sort of conservatory - which gets pretty chilly (we went early Dec) and the meats go cold quickly. Good Service and the set buffet is very good value for money.',
 'categories': ['Restaurant', 'South American Restaurant'],
 'gPlusPlaceId': '109462445253079682250',
 'unixReviewTime': 1354486457,
 'reviewTime': 'Dec 2, 2012',
 'gPlusUserId': '100002870366309555032'}

In [12]:
review1_req = []
for d in review_data:
  if d['gPlusPlaceId'] in place_id_set:
    review1_req.append(d)

In [14]:
review1_req[0]

{'rating': 4.0,
 'reviewerName': 'william spindler',
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'unixReviewTime': 1394669496,
 'reviewTime': 'Mar 12, 2014',
 'gPlusUserId': '100000032416892623125'}

In [ ]:
#with open("USA_reviews1.pkl","wb") as fp:
#  pickle.dump(review1_req,fp)

#with open("USA_reviews1.pkl","rb") as fp:
#  review1_req = pickle.load(fp)

In [15]:
req_places_dict = dict()
for e in req_places:
    if e['gPlusPlaceId'] not in req_places_dict:
        req_places_dict[e['gPlusPlaceId']] = e['gps']

In [16]:
for d in review1_req:
    d['gps'] = req_places_dict[d['gPlusPlaceId']]


In [17]:
review1_req[0]

{'rating': 4.0,
 'reviewerName': 'william spindler',
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'unixReviewTime': 1394669496,
 'reviewTime': 'Mar 12, 2014',
 'gPlusUserId': '100000032416892623125',
 'gps': [40.179159, -122.236162]}

In [18]:
n_train_review1_req = int(0.7 * len(review1_req))
train_review1_req = review1_req[:n_train_review1_req]

n_val_review1_req = int(0.2 * len(review1_req))
val_review1_req = review1_req[n_train_review1_req:n_train_review1_req+n_val_review1_req]

n_test_review1_req = int(0.1 * len(review1_req))
test_review1_req = review1_req[n_train_review1_req+n_val_review1_req:]

In [ ]:
with open("Cat_price_filtered_review_train.pkl","wb") as fp:
    pickle.dump(train_review1_req,fp)

with open("Cat_price_filtered_review_val.pkl","wb") as fp:
    pickle.dump(val_review1_req,fp)

with open("Cat_price_filtered_review_test.pkl","wb") as fp:
    pickle.dump(test_review1_req,fp)

In [ ]:
top_dict = dict()
for d in review1_req:
    if d['categories'] != None:
        k = str(d['categories'])
        if k not in top_dict:
            top_dict[k] = 1
        else:
            top_dict[k] = top_dict[k] + 1

In [ ]:
sorted_top_dict = {k: v for k, v in sorted(top_dict.items(), key=lambda item: item[1], reverse = True)}
sorted_top_list = list(sorted_top_dict.keys())[:50]
sorted_top_list

In [ ]:
with open("Top_Categories_50.pkl","wb") as fp:
    pickle.dump(sorted_top_list,fp)

In [30]:
time_li = [d['unixReviewTime'] for d in review1_req if d['unixReviewTime'] is not None]
max_time = max(time_li)
max_time

1396077471

In [31]:
req_places_dict = dict()
for d in places_data:
  t = d["address"][-1].split()
  for ad in t:
    if ad in state_list:
      req_places_dict[d['gPlusPlaceId']] = ad

In [72]:
review2_req = []
for d in review1_req:
    if d['unixReviewTime'] is not None:
        d['parsed_time'] = d['unixReviewTime']/max_time
        d['state'] = req_places_dict[d['gPlusPlaceId']]
        review2_req.append(d)

In [ ]:
n_train_review2_req = int(0.7 * len(review2_req))
n_train_review2_req

In [ ]:
train_review2_req = review2_req[:n_train_review2_req]
len(train_review2_req)

In [ ]:
with open("Filtered_train.pkl","wb") as fp:
  pickle.dump(train_review2_req,fp)

In [ ]:
#train_review2_req = []
#with open("Filtered_train.pkl","rb") as fp:
#  train_review2_req = pickle.load(fp)

In [76]:
train_review2_req[0]

{'rating': 4.0,
 'reviewerName': 'william spindler',
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'unixReviewTime': 1394669496,
 'reviewTime': 'Mar 12, 2014',
 'gPlusUserId': '100000032416892623125',
 'gps': [40.179159, -122.236162],
 'parsed_time': 0.9989914778876909,
 'state': 'CA'}

In [77]:
with open("Sampled_by_review_count_train_state.pkl","wb") as fp:
    pickle.dump(train_review2_req,fp)

In [ ]:
n_val_review2_req = int(0.2 * len(review2_req))
n_val_review2_req

In [ ]:
val_review2_req = review2_req[n_train_review2_req:n_train_review2_req+n_val_review2_req]
len(val_review2_req)

In [ ]:
#with open("Filtered_val.pkl","wb") as fp:
#  pickle.dump(val_review2_req,fp)

In [80]:
val_review2_req[0]

{'rating': 5.0,
 'reviewerName': 'Michael Papion',
 'reviewText': 'Had a great experience here, took my car in and he had a look at it free of charge and really helped me out. Great customer service and overall knowledge of his trade. I would definitely go to this repair shop again.',
 'categories': ['Auto Repair Shop'],
 'gPlusPlaceId': '116202076999428044236',
 'unixReviewTime': 1379738808,
 'reviewTime': 'Sep 20, 2013',
 'gPlusUserId': '112681887780302676378',
 'gps': [33.832286, -117.959669],
 'parsed_time': 0.9882967361486775,
 'state': 'CA'}

In [81]:
with open("Sampled_by_review_count_val_state.pkl","wb") as fp:
    pickle.dump(val_review2_req,fp)

In [ ]:
n_test_review2_req = int(0.1 * len(review2_req))
n_test_review2_req

In [ ]:
test_review2_req = review2_req[n_train_review2_req+n_val_review2_req:]
len(test_review2_req)

In [ ]:
#with open("Filtered_test.pkl","wb") as fp:
#  pickle.dump(test_review2_req,fp)

In [84]:
test_review2_req[0]

{'rating': 2.0,
 'reviewerName': 'Craig Boswell',
 'reviewText': 'A good company perhaps, but you pay the extra for "my family to yours". Other companies equally qualified all will do it for less.',
 'categories': ['Plumber'],
 'gPlusPlaceId': '101960293183859199370',
 'unixReviewTime': 1366408437,
 'reviewTime': 'Apr 19, 2013',
 'gPlusUserId': '116383650056653356510',
 'gps': [33.4159, -111.793652],
 'parsed_time': 0.9787482896785461,
 'state': 'AZ'}

In [85]:
with open("Sampled_by_review_count_test_state.pkl","wb") as fp:
    pickle.dump(test_review2_req,fp)

In [33]:
price_filtered_places = []
for d in req_places:
    if d['price'] != '' and d['price'] != None:
        price_filtered_places.append(d)

In [34]:
price_filtered_places[0]

{'name': "T C's Referee Sports Bar",
 'price': '$$',
 'address': ['5322 W 26th St', 'Sioux Falls, SD 57106'],
 'hours': [['Monday', [['11:00 am--10:00 pm']]],
  ['Tuesday', [['11:00 am--10:00 pm']], 1],
  ['Wednesday', [['11:00 am--10:00 pm']]],
  ['Thursday', [['11:00 am--10:00 pm']]],
  ['Friday', [['11:00 am--1:00 am']]],
  ['Saturday', [['11:00 am--1:00 am']]],
  ['Sunday', [['11:00 am--10:00 pm']]]],
 'phone': '(605) 361-2208',
 'closed': False,
 'gPlusPlaceId': '100327153115986850675',
 'gps': [43.529494, -96.792244]}

In [ ]:
with open("Price_filtered_places.pkl","wb") as fp:
    pickle.dump(price_filtered_places,fp)

In [20]:
#filtering user data
user_ids = set([d['gPlusUserId'] for d in review1_req])

In [27]:
req_users_info = []
for d in user_data:
  if d['gPlusUserId'] in user_ids:
    req_users_info.append(d)

In [28]:
with open("User_Info_req.pkl","wb") as fp:
  pickle.dump(req_users_info,fp)